In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score

import re
import nltk
import spacy
import string
#import pymorphy2
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords

ru_stopwords = list(stopwords.words("russian"))

# Загрузка и предобработка данных

In [7]:
df = pd.read_csv('../input/some-data1/data.csv')

In [8]:
df

In [9]:
codes = pd.DataFrame(df.Код_подкатегория.value_counts())
codes = codes[codes.Код_подкатегория > 3]
inds = codes.index

In [10]:
def lower_case(x):
    return x.lower()

def remove_symbols(x):
    x = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', x) #deleting newlines and line-breaks
    x = re.sub('[.,:;%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', x) #deleting symbols  
    return x

def remove_punctuation(x):
    PUNCT_TO_REMOVE = string.punctuation
    return x.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(x):
    STOPWORDS = set(stopwords.words('russian'))
    return " ".join([word for word in str(x).split() if word not in STOPWORDS])

def stem_words(text):
    stemmer = SnowballStemmer('russian')
    return " ".join([stemmer.stem(word) for word in text.split()])

def lemmatize_words(x):
    x = " ".join(pymorphy2.MorphAnalyzer().parse(word)[0].normal_form for word in x.split())
    return x

In [11]:
df['preprocessed_text'] = df.Наименование.apply(lambda x : lower_case(x))
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x : remove_symbols(x))
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x : remove_stopwords(x))
#df['preprocessed_text_stemm'] = df['preprocessed_text'].apply(lambda x : stem_words(x))
#df['preprocessed_text_lemant'] = df['preprocessed_text'].apply(lambda x : lemmatize_words(x))
print('done')

In [11]:
df

In [12]:
df['preprocessed_code'] = df.Подкатегория_текст.apply(lambda x : lower_case(x))
df['preprocessed_code'] = df['preprocessed_code'].apply(lambda x : remove_symbols(x))
df['preprocessed_code'] = df['preprocessed_code'].apply(lambda x : remove_stopwords(x))
#df['preprocessed_text_stemm'] = df['preprocessed_text'].apply(lambda x : stem_words(x))
#df['preprocessed_text_lemant'] = df['preprocessed_text'].apply(lambda x : lemmatize_words(x))
print('done')

In [14]:
data = df[df.Код_подкатегория.isin(inds)][['preprocessed_code', 'preprocessed_text', 'Код_подкатегория']]
data.sort_values('Код_подкатегория')

#data = df[['preprocessed_code', 'preprocessed_text', 'Код_подкатегория']]
#data.sort_values('Код_подкатегория')

In [15]:
from tqdm import tqdm

## Создание датасета для обучения модели

In [18]:
data_new = pd.DataFrame()
uniques = data.Код_подкатегория.unique()

for i in tqdm(uniques):
    try :
        pos = df[df.Код_подкатегория == i][['preprocessed_code', 'preprocessed_text']] #.sample(5000)
        pos['Equal'] = 1
    except :
        pos = df[df.Код_подкатегория == i][['preprocessed_code', 'preprocessed_text']]
        pos['Equal'] = 1
        
    data_new = pd.concat([data_new, pos])
    
    for j in np.random.choice(list(set(uniques).difference(set([i]))), 50):
        try: 
            neg = df[df.Код_подкатегория == j][['preprocessed_code', 'preprocessed_text']].sample(15)
        except: 
            neg = df[df.Код_подкатегория == j][['preprocessed_code', 'preprocessed_text']]
        neg['Equal'] = 0
        data_new = pd.concat([data_new, neg])
        

In [19]:
data_new.Equal.value_counts()

In [20]:
data_new = data_new.sample(frac=1).reset_index(drop=True)

In [21]:
data_new

In [58]:
#data_new.to_csv('data_comp.csv', index = False)

In [21]:
data_new.columns

In [84]:
X_tr.columns

# Тренировка модели

In [38]:
catboost_params = {

    'eval_metric': 'Logloss',
    'early_stopping_rounds': 25,
    'use_best_model': True,
    'verbose': 1
}


X_tr, X_val, y_tr, y_val = train_test_split(data_new[['preprocessed_code', 'preprocessed_text']], 
                                            data_new['Equal'], stratify = data_new['Equal'], 
                                            test_size = 0.2)





train_pool = Pool(
    X_tr, 
    y_tr, 
    text_features=['preprocessed_code', 'preprocessed_text'],
    feature_names=['preprocessed_code', 'preprocessed_text']
)
valid_pool = Pool(
    X_val, 
    y_val, 
    text_features=['preprocessed_code', 'preprocessed_text'],
    feature_names=['preprocessed_code', 'preprocessed_text']
)

model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

In [24]:
roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

In [25]:
roc_auc_score(y_tr, model.predict_proba(X_tr)[:, 1])

In [26]:
f1_score(y_val, model.predict(X_val))

In [27]:
from sklearn.metrics import precision_score, recall_score, classification_report

In [28]:
pr = []
rc = []
ac = []
for i in np.linspace(0,1, 20):
    pr.append(precision_score(y_val,  (model.predict_proba(X_val)[:, 1] > i)*1))
    rc.append(recall_score(y_val,  (model.predict_proba(X_val)[:, 1] > i)*1))
    ac.append(accuracy_score(y_val,  (model.predict_proba(X_val)[:, 1] > i)*1))

In [29]:
from matplotlib import pyplot as plt

In [40]:
plt.figure(figsize = (12,7))
plt.plot(np.linspace(0,1, 20), pr, label = 'precision')
plt.plot(np.linspace(0,1, 20), rc, label = 'recall')
plt.plot(np.linspace(0,1, 20), ac, label = 'accuracy')
plt.xlabel('Treshold')
plt.ylabel('Metrics')
#plt.grid()
plt.legend()
plt.show()

In [35]:
print(classification_report(y_val, model.predict(X_val)))

In [36]:
from sklearn.metrics import plot_roc_curve

In [37]:
plot_roc_curve(model, X_val, y_val)

In [62]:
model.save_model('catboost_final_model.cbm')